Import dependencies

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
from collections import defaultdict, Counter
import more_itertools
import heapq
from Bio import Align
from Bio.Align import substitution_matrices
import uuid
import json
from sklearn.model_selection import train_test_split
import time

Load sequence data

In [2]:
with open('../../RSRC/uniref_taxonomy_id_9606_AND_identity_2024_09_13.json') as f:
    human_proteins_json = json.load(f)['results']
    
human_proteins_df = []
for prot in human_proteins_json:
    human_proteins_df.append({'id': prot['id'], 'sequence': prot['representativeMember']['sequence']['value']})
human_proteins_df = pd.DataFrame(human_proteins_df)
human_proteins_df = human_proteins_df[~human_proteins_df['sequence'].str.contains('U')]

df_ds_train, df_ds_test = train_test_split(human_proteins_df, test_size=0.2, random_state=42)

corpus = df_ds_train['sequence']

Extract the initial symbols

In [3]:
alphabet = []
for seq in corpus:
    for letter in seq:
        if letter not in alphabet:
            alphabet.append(letter)
alphabet.sort()
print(alphabet)

['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'X', 'Y']


Import custom classes for efficient BPE

In [4]:
from helper_classes import Sym, SymList, SymPair, MaxHeapMap

Convert every protein sequence into a SymList (a doubly linked list containing tokens as nodes)

In [5]:
sequences = []
for seq in corpus:
    if len(seq) == 0: continue
    symlist = SymList()
    for sym_str in seq:
        symlist.append(Sym(sym_str))
    sequences.append(symlist)


In [6]:
print(sequences[0])

MEVLRRSSVFAAEIMDAFDRCGDAADGLMSSSVWSAQTLASAPTGWWLHSAASAAS


Construct the initial MaxHeapMap

This data structure maintains a max heap of SymbolPairs with respect to their occurence counts.
- A SymbolPair is basically a struct that keeps track of every occurence of a token pair in the corpus

In [7]:
# Generate the data structure for bookkeeping of all symbol pairs found in the data.

# Maps ("A", "B") -> SymPair("A", "B")
pair_database = {}

def add_entry(db, sym1, sym2):
    sym1_str = sym1.literal
    sym2_str = sym2.literal
    curr_pair = db.get((sym1_str, sym2_str), None)
    if curr_pair is None:
        curr_pair = SymPair(sym1_str, sym2_str)
        db[(sym1_str, sym2_str)] = curr_pair
    curr_pair.add_pos((sym1, sym2))


for seq in sequences:
    for sym1, sym2 in more_itertools.pairwise(seq):
        add_entry(pair_database, sym1, sym2)


merge_heap = MaxHeapMap()
for pair in pair_database.values():
    merge_heap.push(pair)

del pair_database

str(merge_heap.peek())

'Pair: (L, L), Count: 153309'

In [ ]:
[str(elem) for elem in merge_heap.heap[:10]]

Define the merging function.

This function is the core of the algorithm.

In [9]:
def merge_pair(sym_pair):
    # print(f"merging: {str(sym_pair)}")
    add_database = {}
    remove_database = {}
    for sym1, sym2 in sym_pair.positions:
        
        # Due to lazy removing, check if we are trying to merge an already removed pair
        if sym1.next.literal != sym_pair.right: continue
        if sym2.prev.literal != sym_pair.left: continue

        merged_sym = Sym(sym1.literal + sym2.literal)
        merged_sym.prev = sym1.prev
        merged_sym.next = sym2.next
        if sym1.prev is not None:
            pre = sym1.prev
            pre.next = merged_sym
            # print(f"Adding new entries to databases", pre, merged_sym)
            # print(f"Adding new entries to databases", pre, sym1)
            add_entry(add_database, pre, merged_sym)
            # Don't attempt to remove currently processed merge pair
            if (pre.literal, sym1.literal) != (sym_pair.left, sym_pair.right): add_entry(remove_database, pre, sym1)
        if sym2.next is not None:
            nex = sym2.next
            nex.prev = merged_sym
            # print(f"Adding new entries to databases", pre, merged_sym)
            # print(f"Adding new entries to databases", pre, sym1)
            add_entry(add_database, merged_sym, nex)
             # Don't attempt to remove currently processed merge pair
            if (sym2.literal, nex.literal) != (sym_pair.left, sym_pair.right): add_entry(remove_database, sym2, nex)
    # print(f"Sizes of the new databases add and remove: {len(add_database)}, {len(remove_database)}")
    for val in add_database.values():
        # print(f"adding: {str(val)}")
        merge_heap.push(val)
    for r_val in remove_database.values():
        # print(f"trying to remove: {str(r_val)}")
        inner_val = merge_heap.remove_by_value(r_val)
        inner_val.count -= r_val.count
        merge_heap.push(inner_val)



Add words by vocabulary size

In [10]:
# # %%prun -s cumulative
# init_vocab = alphabet.copy()

# # Add 10000 words
# for i in range(100):
#     best_pair = merge_heap.pop()
    
#     merge_pair(best_pair)
#     init_vocab.append(best_pair.merged())

# print(init_vocab)

Add words by cutoff frequency threshold

In [11]:
# # %%prun -s cumulative
# init_vocab = alphabet.copy()

# # Add 10000 words
# best_pair = merge_heap.pop()
# max_count = best_pair.count
# while best_pair.count > max_count / 1000:
#     merge_pair(best_pair)
#     init_vocab.append(best_pair.merged())
#     best_pair = merge_heap.pop()

# print(init_vocab)

In [12]:
# [print(str(k)) for k in merge_heap.map]
# print(len(init_vocab))

# cnts = Counter(init_vocab)

## Code blocks related to addition of mutated sequences

In [13]:
# Generates a sorted list of mutations (above the cutoff similarity) for the given sequence.
# Each resulting element is a tuple (m_sequence, score), where:
# - m_sequence: the mutated sequence as string
# - score: similarity score to the original sequence, a number between 0 and 1
# Results are sorted from highest to lowest score.
# First element of the results is the original sequence
# Mutations are based only on substitutions with non-negative matrix values.
def generate_mutations(seq, matrix, cutoff):
    # if len(seq) > 10:
    #     return []
    alp = matrix.alphabet
    candidates = []

    max_score = 0
    for aa in seq:
        if aa != "X":
            max_score += matrix[aa][aa]

    # Create mutation candidates for each symbol in the sequence
    for i, aa in enumerate(seq):
        candidates.append([])
        # Ignore X from calculation
        if aa == "X":
            candidates[i].append((aa, 0.0))
            continue
        # Consider substitutions with non-negative scores
        for c_aa in alp:
            score = matrix[aa][c_aa]
            if score >= -1e-4: # for floating point precision
                similarity_loss = (matrix[aa][aa] - score)/max_score
                # if the similarity loss from this particular aminoacid is large enough
                # to go under the cutoff, don't even consider it
                if similarity_loss < 1 - cutoff:
                    candidates[i].append((c_aa, similarity_loss))

    for idx, candidate in enumerate(candidates):
        candidates[idx] = sorted(candidate, key=lambda x: x[1])

    # Use dfs to search mutations that has score above tbe cutoff, fast.
    def dfs(current_seq, current_score, position):
        # Prune branches where the score is under the cutoff
        if 1-current_score < cutoff:
            return
        
        # If the string is of the required length, save it
        if position == len(seq):
            final_mutations.append((current_seq, 1-current_score))
            return
        
        # Try all candidate mutations at the current position
        for candidate, candidate_score in candidates[position]:
            # Prune if score is under the cutoff
            # This pruning works because candidate scores are sorted
            if 1-(current_score+candidate_score) < cutoff:
                break
            dfs(current_seq+candidate, current_score+candidate_score, position+1)

    final_mutations = []
    dfs("", 0, 0)
    return sorted(final_mutations, key=lambda x: -x[1])

Example usage of generate_mutations

In [14]:
from Bio.Align import substitution_matrices

blosum62 = substitution_matrices.load("BLOSUM62")
pam250 = substitution_matrices.load("PAM250")

In [ ]:
generate_mutations('HMVL', blosum62, 0.8)

An example code block for also adding the mutated sequences during the training of BPE tokenizer

Currently, it adds **all** the mutations that are generated from positive values in the substitution matrix.

In [ ]:
# %%prun -s cumulative
# init_vocab = alphabet.copy()

# Add 100 words + their mutations
for i in range(1000):
    best_pair = merge_heap.pop()
    merge_pair(best_pair)
    merged_string = best_pair.merged()
    init_vocab.append(merged_string)

    # For the mutations:
    
    # Consider only the mutations with a similarity score larger than 0.8
    # [1:] ignores the original string
    mutations = generate_mutations(merged_string, blosum62, 0.8)[1:]
    for mutated_str, score in mutations:
        pairs_to_merge = merge_heap.merged_to_pair.get(mutated_str, [])
        if len(pairs_to_merge) > 0:
            init_vocab.append(mutated_str)
        for pair in pairs_to_merge:
            merge_heap.remove_by_value(pair)
            merge_pair(pair)

print(init_vocab)

In [ ]:
len(init_vocab)

In [ ]:
# For debugging purposes, to see if any duplicates are added
print(len(init_vocab))

cnts = Counter(init_vocab)
cnts

# import json

# with open("test.json", "w") as f:
#     json.dump(list(cnts.keys()), f, indent=4)